# RL Agent traffic light control (SUMO + TraCI + PyTorch)

This notebook trains and evaluates an **RL agent** for traffic control using a simple neural network.

## Algorithm Overview

The RL Agent learning approach:
1. **Brain (Neural Network)**: A small PyTorch network that takes queue state (4 inputs) and outputs action probabilities (2 actions)
2. **State**: Normalized incoming vehicle counts from 4 directions (N, S, E, W)
3. **Action**: Binary choice - phase 0 (N-S green) or phase 2 (E-W green)
4. **Reward**: Negative of average waiting time (lower waiting = higher reward)
5. **Safety**: Yellow phase enforced between transitions

## Workflow
1. **Training Phase**: Learn policy from trial-and-error over multiple episodes
2. **Evaluation Phase**: Test trained agent on fresh scenario
3. **KPI Analysis**: Report metrics compared to baseline

In [ ]:
import torch

# ==========================================
# DEVICE SELECTION (CPU/GPU)
# ==========================================
# Check if CUDA (NVIDIA GPU) is available
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    # Ask user for device selection
    use_gpu = input("\nDo you want to use GPU for training? (y/n): ").strip().lower()
    if use_gpu == 'y':
        device = torch.device('cuda')
        print(f"\n✓ Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("\n✓ Using CPU")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device('cpu')
    print("\n✓ Using CPU")

print(f"\nSelected device: {device}")
print("="*70)

In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import time
import subprocess
import socket

# ==========================================
# 1. NEURAL NETWORK BRAIN
# ==========================================
class TrafficBrain(nn.Module):
    """Simple 2-layer neural network for traffic light control."""
    def __init__(self, input_size=4, hidden_size=32, output_size=2, device='cpu'):
        super(TrafficBrain, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)
    
    def save(self, filepath):
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        torch.save(self.state_dict(), filepath)
        print(f"Model saved to {filepath}")
    
    def load(self, filepath, device='cpu'):
        if os.path.exists(filepath):
            self.load_state_dict(torch.load(filepath, map_location=device))
            print(f"Model loaded from {filepath}")
            return True
        return False


# ==========================================
# 2. CONFIGURATION (RL TRAINING)
# ==========================================
BASE_DIR = os.getcwd()
SCENARIO_DIR = os.path.join(BASE_DIR, "Network with RL control")
RESULTS_DIR = os.path.join(SCENARIO_DIR, "results")
CONFIG_PATH = os.path.join(SCENARIO_DIR, "ff_heterogeneous.sumocfg")
MODEL_DIR = os.path.join(SCENARIO_DIR, "models")
MODEL_PATH = os.path.join(MODEL_DIR, "traffic_agent.pth")

if not os.path.exists(SCENARIO_DIR):
    # Create directory structure if it doesn't exist
    print(f"Creating scenario folder: {SCENARIO_DIR}")
    os.makedirs(SCENARIO_DIR, exist_ok=True)
    # Copy network files from original network
    import shutil
    original_dir = os.path.join(BASE_DIR, "Original network")
    for file in ["ff_heterogeneous.sumocfg", "ff.net.xml", "ff_heterogeneous.rou.xml"]:
        src = os.path.join(original_dir, file)
        dst = os.path.join(SCENARIO_DIR, file)
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied {file}")

if not os.path.exists(RESULTS_DIR):
    print(f"Creating results folder: {RESULTS_DIR}")
    os.makedirs(RESULTS_DIR, exist_ok=True)

if not os.path.exists(CONFIG_PATH):
    sys.exit(f"Config file not found: {CONFIG_PATH}")

# SUMO tools
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

import traci

# SUMO binary resolution (headless mode for faster training)
sumoBinary = "sumo"
if 'SUMO_HOME' in os.environ:
    candidate = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo.exe')
    if os.path.exists(candidate):
        sumoBinary = candidate
    else:
        candidate = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo')
        if os.path.exists(candidate):
            sumoBinary = candidate

if not os.path.exists(sumoBinary):
    try:
        from sumolib import checkBinary
        sumoBinary = checkBinary("sumo")
    except Exception:
        sumoBinary = "sumo"

print(f"Using SUMO binary: {sumoBinary}")

# Logs - save in results folder
sumo_log = os.path.join(RESULTS_DIR, "sumo_log.txt")
sumo_err = os.path.join(RESULTS_DIR, "sumo_err.txt")
traci_stdout = os.path.join(RESULTS_DIR, "traci_stdout.txt")

# Output files - save in results folder
EDGE_DATA_PATH = os.path.join(RESULTS_DIR, "edge_data.xml")


def get_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("localhost", 0))
        return s.getsockname()[1]


# Training parameters
NUM_EPISODES = 1000  # Number of training episodes
EPISODE_DURATION = 3600  # seconds
YELLOW_DURATION = 3  # seconds
MIN_GREEN = 5  # seconds
DISCOUNT_FACTOR = 0.99
LEARNING_RATE = 0.001

TLS_IDS = ["E1", "E2", "E3", "E4"]


def start_sumo(episode_num, mode="train"):
    output_file = os.path.join(RESULTS_DIR, f"tripinfo_{mode}_ep{episode_num}.xml")
    log_handle = open(traci_stdout, "a", encoding="utf-8")
    
    sumoCmd = [
        sumoBinary,
        "-c", CONFIG_PATH,
        "--quit-on-end",
        "--no-step-log",  # Disable step logging for faster execution
        "--tripinfo-output", output_file,
        "--emission-output", os.path.join(RESULTS_DIR, f"emissions_{mode}_ep{episode_num}.xml"),
        "--edgedata-output", os.path.join(RESULTS_DIR, f"edge_data_{mode}_ep{episode_num}.xml"),
        "--log", sumo_log,
        "--error-log", sumo_err,
        "--remote-port", str(get_free_port())
    ]
    
    proc = subprocess.Popen(
        sumoCmd,
        cwd=SCENARIO_DIR,
        stdout=log_handle,
        stderr=log_handle
    )
    return proc, log_handle, sumoCmd[-1]


def connect_traci(port_str, timeout_s=10):
    port = int(port_str)
    deadline = time.time() + timeout_s
    last_error = None
    while time.time() < deadline:
        try:
            return traci.connect(port=port, host="localhost", numRetries=0, waitBetweenRetries=0)
        except Exception as e:
            last_error = e
            time.sleep(0.2)
    raise RuntimeError(f"Could not connect. Last error: {last_error}")


def set_safe_phase(conn, tls_id, target_phase):
    """
    Safely transition to target phase with yellow light.
    Returns: timesteps spent in yellow
    """
    current_phase = conn.trafficlight.getPhase(tls_id)
    if current_phase == target_phase:
        return 0
    
    # Go to yellow phase
    conn.trafficlight.setPhase(tls_id, current_phase + 1)
    for _ in range(YELLOW_DURATION):
        conn.simulationStep()
    
    # Go to target phase
    conn.trafficlight.setPhase(tls_id, target_phase)
    return YELLOW_DURATION


def get_state(conn):
    """
    Get current state: queue lengths on incoming lanes normalized by 50.
    Returns: torch.FloatTensor of shape (1,) on the selected device
    """
    # Count vehicles in queue (halting) on each approach
    lanes = conn.trafficlight.getControlledLanes("E1")
    queue_count = sum(conn.lane.getLastStepHaltingNumber(lane) for lane in lanes)
    
    # Simple state: normalize queue count and move to device
    state = torch.FloatTensor([queue_count / 50.0]).to(device)
    return state


def get_reward(conn):
    """
    Calculate reward based on queue length.
    Reward = -queue_count (we want to minimize queues).
    Lower queue = higher reward.
    """
    lanes = conn.trafficlight.getControlledLanes("E1")
    queue_count = sum(conn.lane.getLastStepHaltingNumber(lane) for lane in lanes)
    
    # Reward is negative of queue count: fewer vehicles waiting = higher reward
    reward = -float(queue_count)
    return reward


def train_rl_agent():
    print("\n" + "="*70)
    print("RL AGENT TRAINING STARTED (HEADLESS MODE)")
    print("="*70)
    print(f"Training {NUM_EPISODES} episodes without GUI visualization")
    print("This will be significantly faster than GUI mode.")
    print(f"Using device: {device}")
    
    # Initialize brain and optimizer
    brain = TrafficBrain(input_size=1, hidden_size=16, output_size=2, device=device).to(device)
    optimizer = optim.Adam(brain.parameters(), lr=LEARNING_RATE)
    
    training_log = []
    
    for episode in range(NUM_EPISODES):
        print(f"\n--- Episode {episode + 1}/{NUM_EPISODES} ---")
        
        proc = None
        log_handle = None
        conn = None
        port_str = None
        
        try:
            proc, log_handle, port_str = start_sumo(episode, mode="train")
            time.sleep(1)  # Reduced wait time for headless mode
            conn = connect_traci(port_str, timeout_s=10)
            
            # Initialize traffic light
            for tls in TLS_IDS:
                conn.trafficlight.setProgram(tls, "0")
            
            episode_reward = 0
            episode_steps = 0
            phase_times = {tls: 0 for tls in TLS_IDS}
            action_history = []
            
            brain.train()
            
            while conn.simulation.getTime() <= EPISODE_DURATION:
                conn.simulationStep()
                sim_time = conn.simulation.getTime()
                
                # Decision every 10 steps (not every step to reduce overhead)
                if int(sim_time) % 10 == 0:
                    state = get_state(conn)
                    reward = get_reward(conn)
                    episode_reward += reward
                    
                    # Forward pass through brain
                    with torch.no_grad():
                        probs = brain(state)
                    
                    # Choose action (0=N-S, 1=E-W)
                    action = torch.argmax(probs).item()
                    target_phase = 0 if action == 0 else 2
                    
                    # Check if enough time in current phase
                    if phase_times["E1"] >= MIN_GREEN:
                        # Apply action
                        set_safe_phase(conn, "E1", target_phase)
                        phase_times["E1"] = 0
                    else:
                        phase_times["E1"] += 1
                    
                    action_history.append(action)
                    
                    # Training step: policy gradient
                    probs = brain(state)  # Recompute for gradient
                    action_prob = probs[action]
                    loss = -torch.log(action_prob + 1e-8) * reward
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                episode_steps += 1
            
            avg_episode_reward = episode_reward / episode_steps if episode_steps > 0 else 0
            training_log.append({
                'episode': episode + 1,
                'avg_reward': avg_episode_reward,
                'total_reward': episode_reward,
                'steps': episode_steps
            })
            
            print(f"Episode {episode + 1} completed: avg_reward = {avg_episode_reward:.4f}")
            
            # Save model periodically
            if (episode + 1) % 50 == 0:
                checkpoint_path = os.path.join(MODEL_DIR, f"traffic_agent_ep{episode + 1}.pth")
                brain.save(checkpoint_path)
            
            if conn is not None:
                conn.close()
            if proc is not None:
                proc.wait(timeout=5)
        
        except Exception as e:
            print(f"Episode {episode + 1} failed: {e}")
        
        finally:
            if conn is not None:
                try:
                    conn.close()
                except:
                    pass
            if proc is not None and proc.poll() is None:
                proc.terminate()
            if log_handle:
                log_handle.close()
    
    # Save trained model
    brain.save(MODEL_PATH)
    
    print("\n" + "="*70)
    print("TRAINING COMPLETED")
    print("="*70)
    
    return brain, training_log


# ==========================================
# 3. EXECUTION
# ==========================================

try:
    brain, training_log = train_rl_agent()
    print(f"\nTraining log summary:")
    for log_entry in training_log:
        print(f"  Episode {log_entry['episode']}: avg_reward = {log_entry['avg_reward']:.4f}")
except Exception as e:
    print(f"Training failed: {e}")

Using SUMO binary: C:\Program Files (x86)\Eclipse\Sumo\bin\sumo-gui.exe

RL AGENT TRAINING STARTED

--- Episode 1/5 ---
Episode 1 completed: avg_reward = -14.3365

--- Episode 2/5 ---
Episode 2 completed: avg_reward = -15.1301

--- Episode 3/5 ---
Episode 3 completed: avg_reward = -13.7541

--- Episode 4/5 ---
Episode 4 completed: avg_reward = -14.7695

--- Episode 5/5 ---
Episode 5 completed: avg_reward = -13.5166
Model saved to c:\Users\antoi\OneDrive\Documents\Documents\Devoirs\Études sup\ENTPE 3A\Majeure transports\Mobility Control and Management\Project\Livrable 2\MOCOM_project_2\Network with RL control\models\traffic_agent.pth

TRAINING COMPLETED

Training log summary:
  Episode 1: avg_reward = -14.3365
  Episode 2: avg_reward = -15.1301
  Episode 3: avg_reward = -13.7541
  Episode 4: avg_reward = -14.7695
  Episode 5: avg_reward = -13.5166


## Evaluation Phase

Test the trained agent on a fresh scenario and compare against baseline.

In [ ]:
# ==========================================
# EVALUATION: Run trained agent
# ==========================================

def evaluate_rl_agent():
    print("\n" + "="*70)
    print("RL AGENT EVALUATION")
    print("="*70)
    print(f"Using device: {device}")
    
    # Load trained model
    brain = TrafficBrain(input_size=1, hidden_size=16, output_size=2, device=device).to(device)
    if not brain.load(MODEL_PATH, device=device):
        print("ERROR: Trained model not found! Run training cell first.")
        return
    
    brain.eval()
    
    proc = None
    log_handle = None
    conn = None
    
    try:
        proc, log_handle, port_str = start_sumo(episode_num=999, mode="eval")
        time.sleep(2)
        conn = connect_traci(port_str, timeout_s=10)
        
        # Initialize traffic light
        for tls in TLS_IDS:
            conn.trafficlight.setProgram(tls, "0")
        
        phase_times = {tls: 0 for tls in TLS_IDS}
        
        print("\nRunning evaluation simulation...")
        
        while conn.simulation.getTime() <= EPISODE_DURATION:
            conn.simulationStep()
            sim_time = conn.simulation.getTime()
            
            # Decision every 10 steps
            if int(sim_time) % 10 == 0:
                state = get_state(conn)
                
                # Forward pass (no gradient needed)
                with torch.no_grad():
                    probs = brain(state)
                    action = torch.argmax(probs).item()
                
                target_phase = 0 if action == 0 else 2
                
                if phase_times["E1"] >= MIN_GREEN:
                    set_safe_phase(conn, "E1", target_phase)
                    phase_times["E1"] = 0
                else:
                    phase_times["E1"] += 1
        
        if conn is not None:
            conn.close()
        if proc is not None:
            proc.wait(timeout=5)
        
        print("Evaluation completed successfully!")
    
    except Exception as e:
        print(f"Evaluation failed: {e}")
    
    finally:
        if conn is not None:
            try:
                conn.close()
            except:
                pass
        if proc is not None and proc.poll() is None:
            proc.terminate()
        if log_handle:
            log_handle.close()


try:
    evaluate_rl_agent()
except Exception as e:
    print(f"Evaluation error: {e}")


RL AGENT EVALUATION
Model loaded from c:\Users\antoi\OneDrive\Documents\Documents\Devoirs\Études sup\ENTPE 3A\Majeure transports\Mobility Control and Management\Project\Livrable 2\MOCOM_project_2\Network with RL control\models\traffic_agent.pth

Running evaluation simulation...
Evaluation completed successfully!


## KPI summary for the RL agent scenario

This section reads the evaluation outputs stored in **Network with RL control/results** and reports key indicators plus comprehensive visualizations.

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASE_DIR = os.getcwd()
SCENARIO_DIR = os.path.join(BASE_DIR, "Network with RL control")
RESULTS_DIR = os.path.join(SCENARIO_DIR, "results")

# Definition of the two directions
DIR_A_EDGES = ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"]  # Forward (E0 -> E5)
DIR_B_EDGES = ["E5E4", "E4E3", "E3E2", "E2E1", "E1E0"]  # Backward (E5 -> E0)

In [ ]:
# ==========================================
# 2. VEHICLE CLASSIFICATION
# ==========================================
def classify_vehicle(veh_id):
    """Classify vehicles based on their ID prefix."""
    prefix = veh_id.split('.')[0] if '.' in veh_id else veh_id
    
    # --- Direction A (E0 -> E5) ---
    if prefix in ['f_7']: 
        return "Bus Dir A (E0->E5)", "Bus", "Dir A"
    
    # --- Direction B (E5 -> E0) ---
    elif prefix in ['f_6']: 
        return "Bus Dir B (E5->E0)", "Bus", "Dir B"
        
    # --- Other categories ---
    elif prefix in ['f_4', 'f_5']:
        return "Other Bus", "Bus", "Other"
    elif prefix in ['f_0', 'f_1', 'f_2', 'f_3']:
        return "Transversal Traffic", "Traffic", "Transversal"
    else:
        return "Background", "Car", "Other"

In [ ]:
# ==========================================
# 3. DATA LOADING
# ==========================================
def load_data(data_folder):
    """Load trip and edge data from XML files (evaluation results)."""
    trip_path = os.path.join(data_folder, "tripinfo_eval_ep999.xml")
    edge_path = os.path.join(data_folder, "edge_data_eval_ep999.xml")

    # --- 1. TRIPINFO (Individual vehicles) ---
    trips = []
    if os.path.exists(trip_path):
        tree = ET.parse(trip_path)
        for t in tree.getroot().findall('tripinfo'):
            cat, vtype, route_group = classify_vehicle(t.get('id'))
            
            duration = float(t.get('duration'))
            route_len = float(t.get('routeLength', 0))
            
            # Calculate average trip speed in km/h
            speed_kmh = (route_len / duration) * 3.6 if duration > 0 else 0

            trips.append({
                'id': t.get('id'),
                'category': cat,
                'type': vtype,
                'route_group': route_group,
                'waitingTime': float(t.get('waitingTime')),
                'duration': duration,
                'routeLength': route_len,
                'speed_kmh': speed_kmh
            })
    df_trips = pd.DataFrame(trips)

    # --- 2. EDGE DATA (Roads) ---
    edges_stats = []
    if os.path.exists(edge_path):
        tree = ET.parse(edge_path)
        intervals = tree.getroot().findall('interval')
        if intervals:
            # Use the last interval
            for e in intervals[-1].findall('edge'):
                eid = e.get('id')
                direction = "None"
                order = 99
                
                if eid in DIR_A_EDGES:
                    direction = "Dir A (E0->E5)"
                    order = DIR_A_EDGES.index(eid)
                elif eid in DIR_B_EDGES:
                    direction = "Dir B (E5->E0)"
                    order = DIR_B_EDGES.index(eid)
                
                if direction != "None":
                    speed_ms = float(e.get('speed', 0))
                    wait_sec = float(e.get('waitingTime', 0)) 
                    
                    edges_stats.append({
                        'edge_id': eid,
                        'direction': direction,
                        'order': order,
                        'speed_kmh': speed_ms * 3.6,        # m/s -> km/h
                        'waiting_hours': wait_sec / 3600.0, # seconds -> hours
                        'density': float(e.get('density', 0))
                    })
    
    df_edges = pd.DataFrame(edges_stats)
    if not df_edges.empty:
        df_edges = df_edges.sort_values(by=['direction', 'order'])

    return df_trips, df_edges

In [ ]:
# ==========================================
# 4. VISUALIZATION
# ==========================================
def create_dashboard(df_trips, df_edges, output_dir):
    """Create comprehensive KPI dashboards."""
    sns.set_theme(style="whitegrid")
    
    # Separate edge datasets by direction
    df_edges_A = df_edges[df_edges['direction'] == "Dir A (E0->E5)"]
    df_edges_B = df_edges[df_edges['direction'] == "Dir B (E5->E0)"]

    # --- FIGURE 1: EDGE ANALYSIS (SPEED & WAIT) ---
    fig1, axes = plt.subplots(3, 2, figsize=(16, 12), constrained_layout=True)
    fig1.suptitle("Edge Analysis: Bidirectional Flow (km/h & Hours)", fontsize=18, fontweight='bold')

    # Column titles
    axes[0,0].set_title("DIRECTION A (E0 -> E5)", fontsize=14, color='green', fontweight='bold')
    axes[0,1].set_title("DIRECTION B (E5 -> E0)", fontsize=14, color='blue', fontweight='bold')

    # ROW 1: SPEED (km/h)
    if not df_edges_A.empty:
        sns.lineplot(data=df_edges_A, x='edge_id', y='speed_kmh', marker='o', color='green', ax=axes[0,0], linewidth=3)
        axes[0,0].set_ylabel("Speed (km/h)")
        for x, y in zip(range(len(df_edges_A)), df_edges_A['speed_kmh']):
            axes[0,0].text(x, y+0.5, f"{y:.1f}", ha='center', color='green', fontweight='bold')
            
    if not df_edges_B.empty:
        sns.lineplot(data=df_edges_B, x='edge_id', y='speed_kmh', marker='o', color='blue', ax=axes[0,1], linewidth=3)
        axes[0,1].set_ylabel("")
        for x, y in zip(range(len(df_edges_B)), df_edges_B['speed_kmh']):
            axes[0,1].text(x, y+0.5, f"{y:.1f}", ha='center', color='blue', fontweight='bold')

    # ROW 2: TOTAL ACCUMULATED WAITING TIME (Hours)
    if not df_edges_A.empty:
        sns.barplot(data=df_edges_A, x='edge_id', y='waiting_hours', hue='edge_id', palette="Greens", ax=axes[1,0], legend=False, edgecolor='black')
        axes[1,0].set_ylabel("Total Accumulated Wait (Hours)")
        
    if not df_edges_B.empty:
        sns.barplot(data=df_edges_B, x='edge_id', y='waiting_hours', hue='edge_id', palette="Blues", ax=axes[1,1], legend=False, edgecolor='black')
        axes[1,1].set_ylabel("")

    # ROW 3: HEATMAPS (Hours)
    if not df_edges_A.empty:
        matrix_A = df_edges_A[['edge_id', 'waiting_hours']].set_index('edge_id').T
        sns.heatmap(matrix_A, annot=True, fmt=".2f", cmap="Reds", ax=axes[2,0], cbar=False)
        axes[2,0].set_xlabel("Edge Sequence")
        
    if not df_edges_B.empty:
        matrix_B = df_edges_B[['edge_id', 'waiting_hours']].set_index('edge_id').T
        sns.heatmap(matrix_B, annot=True, fmt=".2f", cmap="Reds", ax=axes[2,1], cbar_kws={'label': 'Hours'})
        axes[2,1].set_xlabel("Edge Sequence")

    plot_path_1 = os.path.join(output_dir, "edge_analysis_bidirectional.png")
    plt.savefig(plot_path_1, dpi=300)
    print(f"Saved: {plot_path_1}")

    # --- FIGURE 2: BUS vs NETWORK COMPARISON ---
    fig2, axes = plt.subplots(2, 2, figsize=(14, 10), constrained_layout=True)
    fig2.suptitle("Strategic KPI: Bus Performance Analysis", fontsize=16, fontweight='bold')

    # 1. Waiting Time Comparison
    target_buses = df_trips[df_trips['type'] == 'Bus']
    if not target_buses.empty:
        sns.barplot(data=target_buses, x='category', y='waitingTime', hue='category', palette="viridis", ax=axes[0,0], legend=False)
        axes[0,0].set_title("Avg Bus Trip Waiting Time (Seconds)", fontweight='bold')
        axes[0,0].set_ylabel("Seconds per Trip")
        for c in axes[0,0].containers:
            axes[0,0].bar_label(c, fmt='%.1f', padding=3)

    # 2. Speed Comparison
    comp_groups = ["Bus Dir A (E0->E5)", "Bus Dir B (E5->E0)", "Transversal Traffic"]
    speed_df = df_trips[df_trips['category'].isin(comp_groups)]
    
    if not speed_df.empty:
        sns.boxplot(data=speed_df, x='category', y='speed_kmh', hue='category', palette="Set2", ax=axes[0,1], legend=False)
        axes[0,1].set_title("Speed Comparison: Bus vs Cross Traffic", fontweight='bold')
        axes[0,1].set_ylabel("Mean Trip Speed (km/h)")

    # 3. Duration Breakdown - Bottom Left
    dur_df = df_trips.groupby('category')['duration'].mean().reset_index()
    dur_df = dur_df[dur_df['category'].isin(comp_groups + ["Other Bus"])]
    
    if not dur_df.empty:
        sns.barplot(data=dur_df, x='category', y='duration', hue='category', palette="coolwarm", ax=axes[1,0], legend=False)
        axes[1,0].set_title("Average Trip Duration", fontweight='bold')
        axes[1,0].set_ylabel("Duration (seconds)")
        for c in axes[1,0].containers:
            axes[1,0].bar_label(c, fmt='%.1f', padding=3)

    # 4. Text Summary Box - Bottom Right
    axes[1,1].axis('off')
    
    text_str = "KPI SUMMARY REPORT\n" + "="*35 + "\n"
    
    # Global network summary
    text_str += "GLOBAL NETWORK SUMMARY:\n"
    text_str += f"  - Total Vehicles: {len(df_trips)}\n"
    text_str += f"  - Avg Waiting:    {df_trips['waitingTime'].mean():.2f} s\n"
    text_str += f"  - Avg Speed:      {df_trips['speed_kmh'].mean():.2f} km/h\n"
    text_str += "-"*35 + "\n"

    # Category breakdown
    for cat in comp_groups:
        subset = df_trips[df_trips['category'] == cat]
        if not subset.empty:
            avg_wait = subset['waitingTime'].mean()
            avg_speed = subset['speed_kmh'].mean()
            avg_duration = subset['duration'].mean()
            count = len(subset)
            text_str += f"{cat}:\n"
            text_str += f"  - Vehicles:  {count}\n"
            text_str += f"  - Avg Speed: {avg_speed:.2f} km/h\n"
            text_str += f"  - Avg Wait:  {avg_wait:.2f} s\n"
            text_str += f"  - Avg Dur:   {avg_duration:.2f} s\n\n"
    
    axes[1,1].text(0.05, 0.95, text_str, fontsize=10, fontfamily='monospace', verticalalignment='top', 
                   bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))

    plot_path_2 = os.path.join(output_dir, "kpi_analysis_performance.png")
    plt.savefig(plot_path_2, dpi=300)
    print(f"Saved: {plot_path_2}")
    plt.show()

## KPI Analysis Results

This section loads and visualizes the performance data from the RL Agent evaluation (episode 999).

In [ ]:
# ==========================================
# 5. EXECUTION
# ==========================================
print(f"Reading data from: {RESULTS_DIR}")
print(f"Saving plots to: {RESULTS_DIR}")

try:
    trips, edges = load_data(RESULTS_DIR)
    
    if trips.empty:
        print("\nWARNING: No trip data found.")
        print("Please ensure the evaluation simulation has been run.")
        print(f"Expected file: {os.path.join(RESULTS_DIR, 'tripinfo_eval_ep999.xml')}")
    else:
        print(f"\nLoaded {len(trips)} trips and {len(edges)} edge records")
        create_dashboard(trips, edges, RESULTS_DIR)
        print("\nAnalysis completed successfully!")
        
except Exception as e:
    print(f"ERROR: {e}")
    import traceback
    traceback.print_exc()